# Recurrent Neural Networks(RNN)


前面讲的卷积神经网络有一个主要特点,那就是没有记忆。它们单独处理每个输入,在输入与输入之间没有保存任何状态。这种网络叫作前馈网络(feedforward network)，在文本处理时有很大难度，当你在阅读这个句子时,你是一个词一个词地阅读(或者说,眼睛一次扫视一次扫视地阅读),同时会记住之前的内容。这让你能够动态理解这个句子所传达的含义。循环神经网络就可以做到。

循环神经网络(RNN,recurrent neural network)它处理序列的方式是,遍历所有序列元素,并保存一个状态(state),其中包含与已查看内容相关的信息，同一个单元不停地处理不同的输入值，而这些值都是自己产生的。如下图。U是输入层到隐藏层的权重矩阵，o表示输出层的值；V是隐藏层到输出层的权重矩阵。循环神经网络的隐藏层的值s不仅仅取决于当前这次的输入x，还取决于上一次隐藏层的值s。权重矩阵 W就是隐藏层上一次的值作为这一次的输入的权重。右图就是这个网络在t时刻接收到输入$x_t$之后，隐藏层的值是$s_t$ ，输出值是$o_t$ 。关键一点是， $s_t$的值不仅仅取决于$x_t$ ，还取决于$s_{t-1}$ 。

![](.\pic\RNN.png)

我们可以用下面的公式来表示循环神经网络的计算方法：

$O_t=g(V \cdot S_t$)

$S_t=f(U\cdot X_t + W \cdot S_{t-1})$

注意：为了简单说明问题，偏置都没有包含在公式里面。 加上偏置的话是$S_t=f(U\cdot X_t + W \cdot S_{t-1} + b)$，激活函数，一般来说会选择tanh函数。

## LSTM
Long Short Term Memory networks（以下简称LSTMs），一种特殊的RNN网络，该网络设计出来是为了解决长依赖问题。该网络由 Hochreiter & Schmidhuber (1997)引入，并有许多人对其进行了改进和普及。他们的工作被用来解决了各种各样的问题，直到目前还被广泛应用。

所有循环神经网络都具有神经网络的重复模块链的形式。 在标准的RNN中，该重复模块将具有非常简单的结构，例如单个tanh层。标准的RNN网络如下图所示

![](.\pic\LSTM1.png)

LSTMs也具有这种链式结构，但是它的重复单元不同于标准RNN网络里的单元只有一个网络层，它的内部有四个网络层。LSTMs的结构如下图所示。

![](.\pic\LSTM21.png)

在解释LSTMs的详细结构时先定义一下图中各个符号的含义，符号包括下面几种

![](.\pic\LSTM3.png)

图中黄色类似于CNN里的激活函数操作，粉色圆圈表示点操作，单箭头表示数据流向，箭头合并表示向量的合并（concat）操作，箭头分叉表示向量的拷贝操作。

### LSTMs的核心思想

LSTMs的核心是细胞状态（cell state），用贯穿细胞的水平线表示。细胞状态像传送带一样。它贯穿整个细胞却只有很少的分支，这样能保证信息不变的流过整个RNNs。细胞状态如下图所示。

![](.\pic\LSTM4.png)

LSTM网络能通过一种被称为门的结构对细胞状态进行删除或者添加信息。门能够有选择性的决定让哪些信息通过。其实门的结构很简单，就是一个sigmoid层和一个点乘操作的组合。如下图所示。

![](.\pic\LSTM5.png)
因为sigmoid层的输出是0-1的值，这代表有多少信息能够流过sigmoid层。0表示都不能通过，1表示都能通过。

一个LSTM里面包含三个门来控制细胞状态。这三个门分别称为忘记门、输入门和输出门。

LSTM的第一步就是决定细胞状态需要丢弃哪些信息。这部分操作是通过一个称为忘记门的sigmoid单元来处理的。它通过查看$h_{t-1}$和$x_{t}$信息来输出一个0-1之间的向量，该向量里面的0-1值表示细胞状态$C_{t-1}$中的哪些信息保留或丢弃多少。0表示不保留，1表示都保留。忘记门如下图所示。

![](.\pic\LSTM6.png)

下一步是决定给细胞状态添加哪些新的信息。这一步又分为两个步骤，首先，利用$h_{t-1}$和$x_{t}$通过一个称为输入门的操作来决定更新哪些信息。然后利用$h_{t-1}和x_{t}$通过一个tanh层得到新的候选细胞信息$\tilde C_{t}$，这些信息可能会被更新到细胞信息中。这两步描述如下图所示。

![](.\pic\LSTM7.png)

下面将更新旧的细胞信息$C_{t-1}$，变为新的细胞信息$C_{t}$。更新的规则就是通过忘记门选择忘记旧细胞信息的一部分，通过输入门选择添加候选细胞信息$\tilde C_{t}$的一部分得到新的细胞信息$C_{t}$。更新操作如下图所示


![](.\pic\LSTM8.png)

更新完细胞状态后需要根据输入的$h_{t-1}$和$x_{t}$来判断输出细胞的哪些状态特征，这里需要将输入经过一个称为输出门的sigmoid层得到判断条件，然后将细胞状态经过tanh层得到一个-1~1之间值的向量，该向量与输出门得到的判断条件相乘就得到了最终该RNN单元的输出。该步骤如下图所示


![](.\pic\LSTM9.png)

## Tensorflow实现

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

# 输入图片是28*28
n_inputs = 28 #输入一行，一行有28个数据
max_time = 28 #一共28行
lstm_size = 100 #隐层单元
n_classes = 10 # 10个分类
batch_size = 64 #每批次64个样本
n_batch = mnist.train.num_examples // batch_size #计算一共有多少个批次

# 这里的none表示第一个维度可以是任意的长度
x = tf.placeholder(tf.float32,[None,784])
# 正确的标签
y = tf.placeholder(tf.float32,[None,10])

# 输出层初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
# 输出层初始化偏置值
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

# 定义RNN网络
def RNN(X,weights,biases):
    # RNN要求输入数据是三维的格式为[batch_size批次, max_time序列长度, n_inputs数据维度]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    # 定义LSTM
    lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_size)#相当于设置神经元个数
#    final_state是三维数据[state状态, batch_size批次大小, cell.state_size 单元个数100] 最后一个序列的输出值。因为前面序列的预测是不准确的所以不保存。
#    final_state[0]是cell state   Cell的输出ct
#    final_state[1]是hidden_state 序列的输出ht
#    outputs: The RNN output `Tensor`.所有的输出值28个
#       If time_major == False (default), this will be a `Tensor` shaped:
#         `[batch_size, max_time, cell.output_size]`.
#       If time_major == True, this will be a `Tensor` shaped:
#         `[max_time, batch_size, cell.output_size]`.
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)#得到预测值
    return results
    
    
# 计算RNN的返回结果
prediction= RNN(x, weights, biases)  
# 损失函数
loss = tf.losses.softmax_cross_entropy(y,prediction)
# 使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
# 初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))

## Keras 实现

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers.recurrent import LSTM, SimpleRNN

In [ ]:
#Load the data from dataset
(xtrain, ytrain), (xtest, ytest) = keras.datasets.mnist.load_data()

In [ ]:
#Normalizing values
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')
xtrain /= 255
xtest /= 255

In [ ]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

In [ ]:
xtrain[0].shape

In [ ]:
rnn_model = Sequential()

In [ ]:
#定义RNN网络
rnn_model.add(LSTM(100,input_shape=xtrain[0].shape))
rnn_model.add(Dense(units=10))
rnn_model.add(Activation('softmax'))

In [ ]:
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.utils import np_utils
#ytrain to be converted to categorical one hot vectors
ytrain = np_utils.to_categorical(ytrain, 10)
ytest = np_utils.to_categorical(ytest, 10)

In [ ]:
h = rnn_model.fit(xtrain,ytrain,epochs=10,batch_size=64,verbose=2)

In [ ]:
rnn_model.evaluate(xtest,ytest,verbose=2)

In [ ]:
val_loss, val_accuracy = rnn_model.evaluate(xtest,ytest)
print(val_loss)
print(val_accuracy)

In [ ]:
plt.plot(h.epoch,h.history['loss'])
plt.title("Loss")
plt.show()

In [ ]:
plt.plot(h.epoch,h.history['acc'])
plt.title("Accuracy")
plt.show()